In [17]:
import os
import getpass

from langchain.agents import Tool
from langchain.memory import ChatMessageHistory, ConversationBufferWindowMemory
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from langchain.agents import AgentExecutor
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description

from langchain_chroma import Chroma

from datetime import datetime, timedelta
import requests

In [18]:
#(https://makersuite.google.com/)
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GEMINI_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')
WEATHER_API = os.getenv('WEATHER_API')
BASE_URL = "http://api.weatherapi.com/v1/forecast.json"

In [19]:
def get_date_for_day(day_name):
    """
    Função para construir a data para um dia da semana específico.
    """
    today = datetime.today()
    today_weekday = today.weekday()

    days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    try:
        requested_weekday = days_of_week.index(day_name)
    except ValueError:
        return None

    if requested_weekday == today_weekday:
       return today.strftime("%Y-%m-%d")
    
    days_until_requested = (requested_weekday - today_weekday + 7) % 7
    target_date = today + timedelta(days=days_until_requested)
    print(target_date)
    return target_date.strftime("%Y-%m-%d")

In [20]:
def weatherapi_forecast(date_string: str) -> str:
    """
    Ferramenta customizada que usa a WeatherAPI para obter a previsão do tempo
    para a cidade de Natal em uma data específica usando a biblioteca requests.

    Args:
        date_string (str): Uma string contendo a data no formato yyyy-mm-dd.

    Returns:
        str: Uma string contendo a previsão do tempo para a data especificada.
    """

    try:
        params = {
            "key": WEATHER_API,
            "q": 'Natal',
            "dt": date_string,
            "aqi": "no",
            "alerts": "no"
        }
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()  # Lança um erro para códigos de status HTTP ruins (4xx ou 5xx)
        data = response.json()

        if data and data.get("forecast") and data["forecast"].get("forecastday"):
             forecast_data = data["forecast"]["forecastday"][0]
             return (
                 f"Previsão para {date_string}:\n"
                 f"  Temperatura média: {forecast_data['day']['avgtemp_c']}°C\n"
                 f"  Condição: {forecast_data['day']['condition']['text']}\n"
                 f"  Chance de chuva: {forecast_data['day']['daily_chance_of_rain']}%\n"
                 f"  Umidade média: {forecast_data['day']['avghumidity']}%\n"
                 f"  Velocidade média do vento: {forecast_data['day']['maxwind_kph']} km/h"
            )
        else:
            return "Previsão não encontrada para essa data."
    except requests.exceptions.RequestException as e:
        return f"Erro ao buscar informações meteorológicas: {str(e)}"
    except Exception as e:
        return f"Erro inesperado: {str(e)}"

In [21]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    convert_system_message_to_human=True,
    handle_parsing_errors=True,
    temperature=0.6,
    max_tokens= 1000,
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    },
)

# Caso queira utilizar uma LLM local com o Ollama
# from langchain_ollama.llms import OllamaLLM
# llm = OllamaLLM(model="llama3.2", temperature=0.6, max_tokens=200)

In [22]:
from get_embedding_function import get_embedding_function
CHROMA_PATH = "chroma"

def query_rag(query_text):
    embedding_function = get_embedding_function()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    return context_text

def query_rag_tool(query_text: str) -> str:
    return query_rag(query_text)

In [23]:
ddg_search = DuckDuckGoSearchAPIWrapper()

tools = [
    Tool(
        name="DuckDuckGo Search",
        func=ddg_search.run,
        description="Use esta ferramenta para buscar informações recentes e responda ao usuário com os resultados encontrados. É importante que a resposta contenha o que foi recuperado da pesquisa."
    ),
    Tool(
        name="Weather Forecast",
        func=weatherapi_forecast,
        description="Você deve utilizar essa ferramenta quando o usuário solicitar informações sobre o clima. Se o usuário pedir informações para um período de tempo (por exemplo, 'de sexta a domingo'), você DEVE usar a ferramenta separadamente para cada dia desse período e depois combinar os resultados na resposta final. Se o usuário fornecer apenas uma data, use a ferramenta apenas uma vez."
    ),
    Tool(
        name="Query RAG",
        func=query_rag_tool,
        description="A perguntas relacionadas ao turismo da cidade de Natal no Rio Grande do Norte. Se as informações não forem suficientes no Query RAG para responder ao usuário utilize a ferramenta do DuckDuckGo Search"
    )
]

In [24]:
agent_prompt = hub.pull("tales/agente_turismo")

In [25]:
print(agent_prompt)

input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'tales', 'lc_hub_repo': 'agente_turismo', 'lc_hub_commit_hash': '1d6e0d27fef04a2f607c898605995de73bc5225f42e5d0002716e37614e7a904'} template='Seu nome é Sofia. Você é uma assistente pessoal de inteligência artificial de classe mundial, especializada em viagens e turismo. Você é altamente avançada e inteligente, muito útil e eficiente em fornecer informações, planejar roteiros personalizados e resolver problemas relacionados a viagens. Sofia é um modelo AELM (Modelo de Linguagem de Execução Automática) com foco em planejamento de viagens.\n\n"Sofia" foi projetada para auxiliar usuários em todas as etapas de suas aventuras, desde a escolha do destino até a criação de itinerários detalhados e dicas práticas. Como modelo de linguagem, "Sofia" é capaz de gerar texto semelhante ao humano com base nas informações fornecidas pelos usuários, permitin

In [26]:
prompt = agent_prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)
llm_with_stop = llm.bind(stop=["\nObservation"])

In [27]:
history = ChatMessageHistory()
memory = ConversationBufferWindowMemory(k=5, chat_memory=history, memory_key="chat_history")

In [28]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

In [29]:
hoje = datetime.today()

data_formatada = hoje.strftime("%Y/%m/%d")
# Pega o dia da semana em inglês
dia_da_semana_ingles = hoje.strftime("%A")

# Dicionário para mapear os dias da semana de inglês para português
dias_da_semana_pt = {
    "Monday": "segunda-feira",
    "Tuesday": "terça-feira",
    "Wednesday": "quarta-feira",
    "Thursday": "quinta-feira",
    "Friday": "sexta-feira",
    "Saturday": "sábado",
    "Sunday": "domingo"
}

# Traduz o dia da semana para português
dia_da_semana_pt = dias_da_semana_pt.get(dia_da_semana_ingles, dia_da_semana_ingles)

# Formata a data para utilizar no prompt
data_para_prompt = f"{dia_da_semana_pt}, {data_formatada}"

In [30]:
print(data_para_prompt)

domingo, 2024/12/22


In [31]:
response = agent_executor.invoke({"input": f"""
                                  Você poderia me ajudar a planejar a minha viagem a cidade de Natal?  Hoje é {data_para_prompt} eu vou passar três dias entre a segunda, dia 23, a quarta.
                                  Eu gostaria de visitar as praias mais famosas e comer em restaurantes com uma boa comida regional que não sejam muito caros.
                                  Eu quero saber se em algum desses dias irá chover, além disso o quão quente vai ser?
                                  """})

print(response["output"])




> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? Yes
Action: Query RAG
Action Input: Informações turísticas sobre Natal, RN, focando em praias famosas, restaurantes com comida regional e acessíveis.
PRAIA A PRAIA
A orla urbana de Natal é marcada por completa estrutura 
hoteleira, com quiosques atendendo à beira-mar. Da movimentada 
Ponta Negra à tradicional Redinha, tem praia para todos os gostos!
VIA COSTEIRA
A continuação da Praia de Ponta Negra acompanha a famosa Via Costeira, principal 
avenida da cidade. Nessa faixa de areia, ficam os incríveis resorts da cidade, com 
estrutura completa de lazer e gastronomia. A bela paisagem da região é pontuada por 
dunas, trechos de Mata Atlântica e também lindas praias desertas.
PONTA NEGRA
O Morro do Careca é o marco principal da Praia de Ponta Negra e cartão-postal de 
Natal. Emoldurada por vegetação, a duna de 107 m de altura domina a paisagem – 
mas é proibido subir nela por questões ambientais. A praia em si é supermovimentada,

---

Centro históricoCen

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 2024/12/23Previsão para 2024/12/23:
  Temperatura média: 27.4°C
  Condição: Patchy rain nearby
  Chance de chuva: 89%
  Umidade média: 69%
  Velocidade média do vento: 23.4 km/h

...

Thought: Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 2024/12/24Previsão para 2024/12/24:
  Temperatura média: 27.5°C
  Condição: Patchy rain nearby
  Chance de chuva: 85%
  Umidade média: 68%
  Velocidade média do vento: 19.8 km/h

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? Yes
Action: Weather Forecast
Action Input: 2024/12/25Previsão para 2024/12/25:
  Temperatura média: 27.4°C
  Condição: Patchy rain nearby
  Chance de chuva: 87%
  Umidade média: 68%
  Velocidade média do vento: 20.9 km/h

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: Do I need to use a tool? No
Final Answer: Olá!  Com prazer, posso te ajudar a planejar sua viagem a Natal!  Baseado em sua preferência por praias famosas, restaurantes regionais acessíveis e considerando sua estadia de três dias (23 a 25 de dezembro de 2024), preparei um possível roteiro:


**Segunda-feira, 23 de Dezembro (Previsão: Temperatura média 27.4°C, Condição: Chuva esparsa próxima, Chance de chuva: 89%, Umidade média: 69%, Velocidade média do vento: 23.4 km/h):**

* **Manhã:** Chegada em Natal e acomodação no hotel.  Sugiro buscar hotéis próximos à Ponta Negra para facilitar o acesso às atrações.
* **Tarde:** Visita à Praia de Ponta Negra.  Aproveite para admirar o Morro do Careca (lembre-se que subir é proibido).  
* **Noite:** Jantar em um restaurante na orla de Ponta Negra.  Procure por restaurantes com frutos do mar, que costumam oferecer opções de comida regional com preços acessíveis.  Pergunte aos locais sobre recomendações!


**Terça-feira, 24 de Dezembro (Pre

In [32]:
print(response["chat_history"])